# Prelude
Hehe I really liked this activity.

# Acknowledgements
I'd like to thank my instructors, Sir Rene Principe Jr. and Sir Kenneth Leo, for guiding me throughout the activity. I would also like to thank my professor, Ma'am Jing, for guiding me in my coding while my classmates and I worked in R202. I would also like to acknowledge my classmates: Abdel, Johnenn, Jonabel, Richmond, Lovely, Hans, Genesis, Jeruine, Rusher, and Ron for helping me complete this activity.

# Code starts Here

In [181]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import statistics as st
from PIL import Image
from skimage.measure import label, regionprops, regionprops_table
import pandas as pd
from scipy.stats import gaussian_kde
import os
from scipy.ndimage.morphology import binary_fill_holes

image_repository = "./Image repository/"
data_repository = "./Features extracted/"

# initializing this so I have a placeholder
image = 0
R,G,B, ave_R,ave_G, ave_B = None, None, None, None, None, None

C:\Users\Julian Maypa\AppData\Local\Temp\ipykernel_19292\2692339592.py:10: DeprecationWarning: Please use `binary_fill_holes` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import binary_fill_holes


> #### ${\tt thresholding(image, stdev\_mult):}$
> 1. Accepts a grayscale image
> 2. Gets the median and standard deviation of the image.
> 3. Binarizes the image by zeroing out the values that are in the range [median - ${\tt stdev\_mult}\times$(stdev), median + ${\tt stdev\_mult}\times$(stdev)] and setting the other values to $1$.
> 4. Returns the binarized image.<br>
> $~$

In [91]:
def thresholding(image, stdev_mult):

    # I'll automate the thresholding. Since we know that the highest peak in the histogram represents the image's background, 
    # my left and right borders will be np.median(image) -+ stdev_mult*stdev respectively.
    peak_ = np.median(image.flatten())
    stdev_ = np.std(image.flatten())

    # Now setting up the borders
    left_border = int(np.floor(peak_ - stdev_mult*stdev_))
    right_border = int(np.ceil(peak_ + stdev_mult*stdev_))

    # Converting to binary through. The values to the left of the left border and to the right of the right boarder are set to 1.
    T, thresholded = cv2.threshold(image, left_border, right_border, cv2.THRESH_BINARY_INV)

    return thresholded


> #### ${\tt morphing\_time(filename, filetype, commands, record = False, ROI\_size = None,}$ 
> #### ${\tt blob\_numbers = False, bounding\_box = False):}$
> 1. opens the image with the stated ${\tt filename}$.
> 2. Loops through a list of ${\tt commands}$ which contains the morphological operations along with their parameters.
> 3. Displays the original image and the final processed image.
> 4. If ${\tt record}$ is ${\tt True}$, then the area, centroid, bounding box, perimeter, eccentricity, minor axis, and major axis of the detected objects are saved to a csv file
> 5. If ${\tt ROI\_size}$ is ${\tt True}$, then the measurements are converted to $\rm{cm}$ or whatever unit you want to set.
> 6. If ${\tt blob\_numbers}$ is ${\tt True}$, then a number is also displayed on the labelled objects.
> 6. If ${\tt bounding\_box}$ is ${\tt True}$, then the detected objects in the original image is enclosed in a box.
> $~$

In [182]:
def morphing_time(filename, commands, plotting = False):
    # Setting initial parameters
    cmap_color = "gray"
    image = cv2.imread(image_repository + filename)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    original_image = image.copy()

    # This is where the commands are looped through
    for i in range(len(commands)):
        process = commands[i][0].lower()

        match process:
            
            case 'grayscale':       
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            case 'thresholding':
                image = thresholding(image, commands[i][1])
            case 'dilation':
                image = cv2.dilate(image, commands[i][1], iterations = commands[i][2])
            case 'erosion':
                image = cv2.erode(image, commands[i][1], commands[i][2])
            case 'opening':
                image = cv2.morphologyEx(image, cv2.MORPH_OPEN, commands[i][1])
            case 'closing':
                image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, commands[i][1])
            case 'median_blur':
                image = cv2.medianBlur(image, commands[i][1])
            case 'simple_binarize':
                # All pixels beow the boundary ares set to 0 and above are set to 1.
                boundary = np.percentile(image, commands[i][1])
                image[image <= boundary] = 0
                image[image > boundary] = 1  
            case 'invert':
                image= cv2.bitwise_not(image)
            case 'apply_mask':
                mask_2D = image.copy()
                mask = np.dstack([image,image,image])
                image = original_image*mask
            case 'separate_rgb':
                R,G,B = cv2.split(image)
            case 'average_rgb':
                ave_R = np.average(R[mask_2D == 1])
                ave_G = np.average(G[mask_2D == 1])
                ave_B = np.average(B[mask_2D == 1])
            case 'get_mask':
                image = mask_2D
            case 'save':
                labelled_image = label(image)
                regions = regionprops(labelled_image)
                props = regionprops_table(labelled_image, properties=commands[i][1])
                data = pd.DataFrame(props)
                data.drop(data[data['area'] <commands[i][2]].index, inplace=True)
            case 'fill_holes':
                image = binary_fill_holes(image)
            case 'done':
                break
                
            case other:
                print(commands[i][0] + " did nothing lol.")


    # For graphing
    if plotting == True:
        fig, ax = plt.subplots(1,2, figsize = (12,8))
        ax[0].imshow(original_image)
        ax[0].set_title(filename)

        ax[1].imshow(image, cmap = cmap_color)
        ax[1].set_title("Processed Image")

        plt.tight_layout()
        plt.show()
    return data, ave_R, ave_G, ave_B


# Outputs

In [189]:
overall_data = pd.DataFrame(columns = ['area', 'eccentricity', 'ave_R', 'ave_G', 'ave_B'])
commands = [
            ['invert'],
            ['median_blur', 11],
            ['grayscale'],
            ['simple_binarize', 57.5],
            ['fill_holes'],
            ['apply_mask'],
            ['separate_RGB'],
            ['average_rgb'],
            ['get_mask'],
            ['save', ('area','centroid', 'bbox', 
                      'perimeter', 'eccentricity' , 
                      'orientation','axis_major_length',
                        'axis_minor_length'), 1000 ]
        ]
for images in os.listdir(image_repository + 'Banana/'):
 
    # check if the image ends with png
    if (images.endswith(".jpg")):
      data, ave_R, ave_G, ave_B = morphing_time('Banana/'+images, commands)
      new_data = [data.iloc[0]['area'], data.iloc[0]['eccentricity'], ave_R, ave_G, ave_B]
      overall_data.loc[len(overall_data)] = new_data

overall_data.to_csv(data_repository + "Banana data.csv") 

      
      

In [197]:
overall_data = pd.DataFrame(columns = ['area', 'eccentricity', 'ave_R', 'ave_G', 'ave_B'])
commands = [
            ['invert'],
            ['median_blur', 15],
            ['grayscale'],
            ['simple_binarize', 27],
            ['fill_holes'],
            # ['closing', cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))],
            ['apply_mask'],
            ['separate_RGB'],
            ['average_rgb'],
            ['get_mask'],
            ['save', ('area','centroid', 'bbox', 
                      'perimeter', 'eccentricity' , 
                      'orientation','axis_major_length',
                        'axis_minor_length'), 2000 ]
        ]
for images in os.listdir(image_repository + 'Apple/'):
 
    # check if the image ends with png
    if (images.endswith(".jpg")):
      data, ave_R, ave_G, ave_B = morphing_time('Apple/'+images, commands)
      new_data = [data.iloc[0]['area'], data.iloc[0]['eccentricity'], ave_R, ave_G, ave_B]
      overall_data.loc[len(overall_data)] = new_data

overall_data.to_csv(data_repository + "Apple data.csv") 